In [31]:
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect

import numpy as np


In [3]:
engine = create_engine("sqlite:///datasets/belly_button_biodiversity.sqlite")

# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

# Save references to each table
otu = Base.classes.otu
samples = Base.classes.samples
samples_meta = Base.classes.samples_metadata

# Create our session (link) from Python to the DB
session = Session(engine)


In [5]:
inspector = inspect(engine)

In [6]:
inspector.get_table_names()

['otu', 'samples', 'samples_metadata']

In [7]:
columns = inspector.get_columns('otu')
for column in columns:
    print(column["name"], column["type"])

otu_id INTEGER
lowest_taxonomic_unit_found TEXT


In [44]:
columns = inspector.get_columns('samples_metadata')
for column in columns:
    print(column["name"], column["type"])

SAMPLEID INTEGER
EVENT TEXT
ETHNICITY TEXT
GENDER TEXT
AGE INTEGER
WFREQ INTEGER
BBTYPE TEXT
LOCATION TEXT
COUNTRY012 TEXT
ZIP012 INTEGER
COUNTRY1319 TEXT
ZIP1319 INTEGER
DOG TEXT
CAT TEXT
IMPSURFACE013 INTEGER
NPP013 FLOAT
MMAXTEMP013 FLOAT
PFC013 FLOAT
IMPSURFACE1319 INTEGER
NPP1319 FLOAT
MMAXTEMP1319 FLOAT
PFC1319 FLOAT


In [27]:
#app.route('/names')
columns = inspector.get_columns('samples')
names = []
for column in columns:
    names.append(column['name'])
del names[0]
#return jsonify(names)

In [42]:
#app.route('/otu')
results = session.query(otu.lowest_taxonomic_unit_found).all()
descriptions = [result[0] for result in results]
#return jsonify(descriptions)

In [ ]:
#@app.route('/metadata/<sample>')
def sample_metadata(sample):
    sample_id = sample.replace('BB_','')
    result = session.query(samples_meta).filter(samples_meta.SAMPLEID == sample_id).first()
    metadata = {
        'AGE':result.AGE,
        'BBTYPE':result.BBTYPE,
        'ETHNICITY':result.ETHNICITY,
        'GENDER':result.GENDER,
        'LOCATION':result.LOCATION,
        'SAMPLEID':result.SAMPLEID
    }
    return jsonify(metadata)